#### 1)	참석자 명단과 명찰양식 준비하고 로드하기

In [34]:
import six
import copy
import pandas as pd
from pptx import Presentation
from pptx.util import Pt

df = pd.read_excel("행사 참석자 명단.xlsx")
prs = Presentation("명찰 양식.pptx")

#### 2) 슬라이드 복사 사용자 함수 만들기

In [35]:
def duplicate_slide(prs, org_slide):
    #add_slide() 메서드를 이용해 새로운 슬라이드 생성한 후 개체를 복사해 옴
    copied_slide = prs.slides.add_slide(org_slide.slide_layout)
    for shape in org_slide.shapes:
        org_elment = shape.element
        new_element = copy.deepcopy(org_elment)
        copied_slide.shapes._spTree.insert_element_before(new_element, "p:extLst")
    for _, value in six.iteritems(org_slide.part.rels):
        if "notesSlide" not in value.reltype:
            copied_slide.part.rels.add_relationship(
                value.reltype,
                value._target,
                value.rId
            )
    return copied_slide

#### 3) 명찰 슬라이드 제작

In [36]:
from pptx.util import Pt

nametag_count = 0
for person_count in range(len(df)): #참석자 수(117명) 만큼 반복문 실행
    #슬라이드 한장당 4개의 명찰이 있으므로
    #현재 반복문이 실행되고 있는 person_count를 4로 나누고 남은 값이
    #0일 때(=4의 배수일 때) 슬라이드를 복제하고 데이터를 입력함
    if person_count % 4 == 0:
        new_slide = duplicate_slide(prs, prs.slides[0]) #첫 번째 슬라이드를 복제하여 추가
        for shape in new_slide.shapes: #슬라이드에 있는 개체를 체크하여 데이터 입력
            #명찰 입력이 완료된 갯수(nametag_count)가
            #명단 데이터 갯수(len(df))보다 적을 때에만 실행
            if nametag_count < len(df):
                if shape.shape_type == 17 and shape.text == "소속":
                    shape.text_frame.paragraphs[0].text = df.iloc[nametag_count, 1]
                    shape.text_frame.paragraphs[0].font.size = Pt(24)
                    shape.text_frame.paragraphs[0].font.bold = True
                    shape.text_frame.paragraphs[0].font.name = "맑은 고딕"
                elif shape.shape_type == 17 and shape.text == "이름":
                    shape.text_frame.paragraphs[0].text = df.iloc[nametag_count, 0]
                    shape.text_frame.paragraphs[0].font.size = Pt(60)
                    shape.text_frame.paragraphs[0].font.bold = True
                    shape.text_frame.paragraphs[0].font.name = "맑은 고딕"
                    #이름입력까지 완료되면 입력 완료된 갯수(nametag_count) 업데이트
                    nametag_count += 1

prs.save("명찰 자동제작 결과.pptx") #새로운 파일로 저장

#### 사전준비) 가짜 데이터 생성

In [9]:
import random
from korean_name_generator import namer
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Side, Border, Alignment

name_list = []
depart_list= ["재무팀", "마케팅팀", "인사팀", "교육팀", "노무팀", "홍보팀", "대외협력팀", "보안관리팀"]

# 남자 이름 생성
for name in range(0, 117):
    rand_no = random.randrange(0, len(depart_list))
    name_list.append({"이름" : namer.generate(True), "소속" : depart_list[rand_no]})
df = pd.DataFrame(name_list)
display(df)
df.to_excel("행사 참석자 명단.xlsx", index=False)

wb = load_workbook("행사 참석자 명단.xlsx")
ws = wb["Sheet1"]

fill_style = PatternFill(fill_type="solid", start_color="fcba03") #셀 배경색 설정 객체 생성
font_style_header = Font(color="FFFFFF", sz=12, bold=True) #폰트 설정 객체 생성
font_style = Font(color="000000", sz=12, bold=False) #폰트 설정 객체 생성
side_style = Side(color="CCCCCC", border_style="thin")
alignment_style = Alignment(horizontal="center", vertical="center",)

for row in ws:
    for col in row:
        col.border = Border(
            top = side_style,
            bottom = side_style,
            left = side_style,
            right = side_style
        )
        col.alignment = alignment_style
        if col.row == 1:
            col.fill = fill_style
            col.font = font_style_header
        else:
            col.font = font_style

ws.column_dimensions["A"].width = 15
ws.column_dimensions["B"].width = 15
            
wb.save(filename = "행사 참석자 명단.xlsx")

,이름,소속
0,황호재,홍보팀
1,윤재본,재무팀
2,최미윤,인사팀
3,곽태균,노무팀
4,나조연,노무팀
...,...,...
112,정지하,대외협력팀
113,박승군,재무팀
114,김주수,재무팀
115,지청슨,대외협력팀
